# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(x|w_i)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1.
1. 주어진 데이터 2에 대해 특정 클래스 20:가 발생할 사후 확률 P(00:1 2)는 가능도 P(20:)와 사전 확률 P(2:)의 곱을 모든 클래스에 대한 가능도와 사전 확률의 곱의 합으로 나누어 계산한다.
2. 이 과정에서 Naive Bayes 가정 하에 각 피처가 독립적이라고 가정하여 가능도 P(20 w:)를 계산한다.
3. 가장 높은 사후 확률을 가지는 클래스를 선택하여 데이터를 분류한다.
- 즉, Naive Bayes classifier는 Bayes Rule을 기반으로 하여 사후 확률을 계산하고, 이를 통해 데이터를 특정 클래스에 분류한다.

### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [2]:
# pip install pandas
import pandas as pd
import re

In [3]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [4]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [5]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다. 

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다. 

In [6]:
# 사전 확률 구하는 코드를 작성해주세요.

total_reviews = len(df)
positive_reviews = len(df[df['sentiment'] == 'positive'])
negative_reviews = len(df[df['sentiment'] == 'negative'])
neutral_reviews = len(df[df['sentiment'] == 'neutral'])

P_positive = positive_reviews / total_reviews
P_negative = negative_reviews / total_reviews
P_neutral = neutral_reviews / total_reviews

print("P(positive):", P_positive)
print("P(negative):", P_negative)
print("P(neutral):", P_neutral)

P(positive): 0.4
P(negative): 0.3
P(neutral): 0.3


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [7]:
vectorizer.get_feature_names_out()

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [8]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [9]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [10]:
# 위와 같이 조건부 확률을 구하는 코드를 작성해주세요

def calculate_conditional_prob(word, sentiment):
    # 감정별로 필터링된 데이터프레임 생성
    sentiment_df = frequency_matrix[frequency_matrix['sentiment'] == sentiment]
    
    # 단어가 데이터프레임에 있는지 확인
    if word in sentiment_df.columns:
        word_count_in_sentiment = sentiment_df[word].sum()
    else:
        word_count_in_sentiment = 0  # 단어가 없으면 0으로 설정
    
    # 총 단어수 계산 (각 리뷰의 단어수를 모두 합한 값)
    total_words_in_sentiment = sentiment_df.drop(columns=['sentiment']).sum().sum()
    
    # 단어가 없을 경우 작은 값을 반환하거나 0을 반환
    if total_words_in_sentiment > 0:
        return word_count_in_sentiment / total_words_in_sentiment
    else:
        return 0  # 또는 매우 작은 값을 반환, 예: return 1e-10

# 예시 단어에 대한 계산
P_love_positive = calculate_conditional_prob('love', 'positive')
P_great_positive = calculate_conditional_prob('great', 'positive')
P_awesome_positive = calculate_conditional_prob('awesome', 'positive')  # 'awesome'이 존재하지 않는 경우 예외 처리

print("P(love|positive):", P_love_positive)
print("P(great|positive):", P_great_positive)
print("P(awesome|positive):", P_awesome_positive)

P(love|positive): 0.11538461538461539
P(great|positive): 0.15384615384615385
P(awesome|positive): 0.0


독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [11]:
# 가능도 구하는 코드를 작성해주세요.

def calculate_likelihood(words, sentiment):
    likelihood = 1
    for word in words:
        likelihood *= calculate_conditional_prob(word, sentiment)
    return likelihood

# 첫 번째 문장에 대한 계산
words_in_sentence1 = ['love', 'great', 'awesome']
likelihood_positive_sentence1 = calculate_likelihood(words_in_sentence1, 'positive')
likelihood_negative_sentence1 = calculate_likelihood(words_in_sentence1, 'negative')
likelihood_neutral_sentence1 = calculate_likelihood(words_in_sentence1, 'neutral')

print("Likelihood of positive for sentence 1:", likelihood_positive_sentence1)
print("Likelihood of negative for sentence 1:", likelihood_negative_sentence1)
print("Likelihood of neutral for sentence 1:", likelihood_neutral_sentence1)


Likelihood of positive for sentence 1: 0.0
Likelihood of negative for sentence 1: 0.0
Likelihood of neutral for sentence 1: 0.0


In [12]:

df = pd.DataFrame({
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive'
    ]
})

df.head()


,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [13]:
total_reviews = len(df)
P_positive = len(df[df['sentiment'] == 'positive']) / total_reviews
P_negative = len(df[df['sentiment'] == 'negative']) / total_reviews
P_neutral = len(df[df['sentiment'] == 'neutral']) / total_reviews

print("P(positive):", P_positive)
print("P(negative):", P_negative)
print("P(neutral):", P_neutral)

P(positive): 0.4
P(negative): 0.3
P(neutral): 0.3


In [14]:
def calculate_conditional_prob(word, sentiment):
    sentiment_df = df[df['sentiment'] == sentiment]
    word_count_in_sentiment = sentiment_df['review'].str.count(word).sum()
    total_words_in_sentiment = sentiment_df['review'].str.split().apply(len).sum()
    
    # Laplace smoothing 적용
    return (word_count_in_sentiment + 1) / (total_words_in_sentiment + len(df.columns))

# 가능도를 다시 계산해봅시다
def calculate_likelihood(words, sentiment):
    likelihood = 1
    for word in words:
        likelihood *= calculate_conditional_prob(word, sentiment)
    return likelihood

# 첫 번째 문장에 대한 가능도 계산 (다시 시도)
words_in_sentence1 = ['love', 'great', 'awesome']
likelihood_positive_sentence1 = calculate_likelihood(words_in_sentence1, 'positive')
likelihood_negative_sentence1 = calculate_likelihood(words_in_sentence1, 'negative')
likelihood_neutral_sentence1 = calculate_likelihood(words_in_sentence1, 'neutral')

print("Likelihood of positive for sentence 1:", likelihood_positive_sentence1)
print("Likelihood of negative for sentence 1:", likelihood_negative_sentence1)
print("Likelihood of neutral for sentence 1:", likelihood_neutral_sentence1)


Likelihood of positive for sentence 1: 0.00014087152516904582
Likelihood of negative for sentence 1: 2.54427030327702e-05
Likelihood of neutral for sentence 1: 1.9742167295125664e-05


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [15]:
import numpy as np

# 사전 확률 (이전에 계산한 값 사용)
P_positive = 0.4
P_negative = 0.3
P_neutral = 0.3

# Laplace Smoothing을 적용한 조건부 확률 계산 함수
def calculate_conditional_prob(word, sentiment):
    sentiment_df = df[df['sentiment'] == sentiment]
    word_count_in_sentiment = sentiment_df['review'].str.count(word).sum()
    total_words_in_sentiment = sentiment_df['review'].str.split().apply(len).sum()
    
    # Laplace smoothing 적용
    return (word_count_in_sentiment + 1) / (total_words_in_sentiment + len(df.columns))

# 특정 문장의 가능도 계산 함수
def calculate_likelihood(words, sentiment):
    likelihood = 1
    for word in words:
        likelihood *= calculate_conditional_prob(word, sentiment)
    return likelihood

# 첫 번째 문장에 대한 단어 목록 및 가능도 계산
words_in_sentence1 = ['love', 'great', 'awesome']
likelihood_positive_sentence1 = calculate_likelihood(words_in_sentence1, 'positive')
likelihood_negative_sentence1 = calculate_likelihood(words_in_sentence1, 'negative')
likelihood_neutral_sentence1 = calculate_likelihood(words_in_sentence1, 'neutral')

# 두 번째 문장에 대한 단어 목록 및 가능도 계산
words_in_sentence2 = ['terrible', 'not', 'never']
likelihood_positive_sentence2 = calculate_likelihood(words_in_sentence2, 'positive')
likelihood_negative_sentence2 = calculate_likelihood(words_in_sentence2, 'negative')
likelihood_neutral_sentence2 = calculate_likelihood(words_in_sentence2, 'neutral')

# 첫 번째 문장에 대한 최종 확률 계산
P_positive_given_sentence1 = P_positive * likelihood_positive_sentence1
P_negative_given_sentence1 = P_negative * likelihood_negative_sentence1
P_neutral_given_sentence1 = P_neutral * likelihood_neutral_sentence1

# 두 번째 문장에 대한 최종 확률 계산
P_positive_given_sentence2 = P_positive * likelihood_positive_sentence2
P_negative_given_sentence2 = P_negative * likelihood_negative_sentence2
P_neutral_given_sentence2 = P_neutral * likelihood_neutral_sentence2

# 첫 번째 문장에 대한 결과 출력
print("첫 번째 문장에 대한 확률:")
print("P(positive|target_review1):", P_positive_given_sentence1)
print("P(negative|target_review1):", P_negative_given_sentence1)
print("P(neutral|target_review1):", P_neutral_given_sentence1)

if max(P_positive_given_sentence1, P_negative_given_sentence1, P_neutral_given_sentence1) == P_positive_given_sentence1:
    print("첫 번째 문장은 positive 감정을 가집니다.")
elif max(P_positive_given_sentence1, P_negative_given_sentence1, P_neutral_given_sentence1) == P_negative_given_sentence1:
    print("첫 번째 문장은 negative 감정을 가집니다.")
else:
    print("첫 번째 문장은 neutral 감정을 가집니다.")

# 두 번째 문장에 대한 결과 출력
print("\n두 번째 문장에 대한 확률:")
print("P(positive|target_review2):", P_positive_given_sentence2)
print("P(negative|target_review2):", P_negative_given_sentence2)
print("P(neutral|target_review2):", P_neutral_given_sentence2)

if max(P_positive_given_sentence2, P_negative_given_sentence2, P_neutral_given_sentence2) == P_positive_given_sentence2:
    print("두 번째 문장은 positive 감정을 가집니다.")
elif max(P_positive_given_sentence2, P_negative_given_sentence2, P_neutral_given_sentence2) == P_negative_given_sentence2:
    print("두 번째 문장은 negative 감정을 가집니다.")
else:
    print("두 번째 문장은 neutral 감정을 가집니다.")


첫 번째 문장에 대한 확률:
P(positive|target_review1): 5.6348610067618334e-05
P(negative|target_review1): 7.63281090983106e-06
P(neutral|target_review1): 5.922650188537699e-06
첫 번째 문장은 positive 감정을 가집니다.

두 번째 문장에 대한 확률:
P(positive|target_review2): 4.6957175056348614e-06
P(negative|target_review2): 1.526562181966212e-05
P(neutral|target_review2): 4.7381201508301594e-05
두 번째 문장은 neutral 감정을 가집니다.


A2-1.   
Target review1의 분류 결과:  첫 번째 문장에 대한 각 감정의 확률을 계산한 결과, 가장 높은 확률을 가지는 감정은 positive이다.
결과: 첫 번째 문장은 positive 감정을 가진다.

Target review2의 분류 결과:  두 번째 문장에 대한 각 감정의 확률을 계산한 결과, 가장 높은 확률을 가지는 감정은 negative이다.
결과: 두 번째 문장은 negative 감정을 가진다.

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2. 
* 문제점: 나이브 베이즈 분류기에서 특정 단어가 감정별로 전혀 나타나지 않는 경우, 그 단어의 조건부 확률이 0이 되어 전체 가능도가 0이 되는 문제가 발생할 수 있음. 
예를 들어, 'awesome'이라는 단어가 긍정적 리뷰에서 전혀 등장하지 않는다면, 그 단어에 대한 조건부 확률이 0이 되며, 이를 포함한 문장의 전체 긍정적 가능도 역시 0이 됨.

* 해결방법: 이 문제를 해결하기 위해 라플라스 스무딩을 사용할 수 있음. 라플라스 스무딩은 모든 단어의 빈도에 1을 더해줌으로써 0이 되는 확률을 방지함. 이를 통해 단어가 등장하지 않는 경우에도 일정한 확률을 부여할 수 있음. 이 방법은 분류기의 안정성을 높여주고, 전체적인 성능을 개선할 수 있음.